# Generator basics

This is a somewhat extensive explanation for the advanced Python developer. Normally, one will not have to dive into these details, unless complex macro structures are created. You might want to look at the simpler [Thumb rules](#Thumb-rules) instead.

CyriteHDL is internally driven by generator machinery, rather than  particular processing into a target representation by AST translation.

When writing extension functions, it is important to know the generator internals, as the rules apply: You don't always get what you see (due to internal translation).

Take a simple conditional function in native Python:

In [1]:
def native(a, b, z):
    if a == 1:
        b.next = 2
        z.next = True
    else:
        b.next = 0
        z.next = False

When this function is run, it determines at runtime when to branch, and not visit code inside condition statements that are not met. In order to translate this code to a target, a different internal representation is more practical:

In [2]:
def generator_func(context, a, b, z):
    yield (
        context.If(a == 1).Then(
            b.set(2), z.set(True)
        ).Else(
            b.set(0), z.set(False)
        )
    )

A specific caller is able to extract the entire branch tree from this construct, given a `context.If` class hierarchy that manages it. We create a simple context ad hoc:

In [3]:
from myirl.kernel.struct_cond import If

class AdHocContext:
    If = If

Then we iterate through the generation function and print out the statements in their debug-style representation:

In [4]:
from myirl import *

a, b = [ Signal(intbv()[6:], name = n) for n in "ab" ]
z = Signal(bool(), name = 'z')

context = AdHocContext()

def walk(context, func, emit, *args):
    for g in func(context, *args):
        emit(g)

def _emit(g):
    if isinstance(g, CondIterable):
        for s in reversed(list(g)):
            print(s)
        
walk(context, generator_func, _emit, a, b, z)

If (a == C:1)
Then {(b <= 2, z <= B:true)}
Then {(b <= 0, z <= B:false)}


**Note**: The `Else` branch lists also as `Then` in the internal representation.

A more complex target is a HDL language target instanced as follows:

In [5]:
from myirl.targets.dummy import DummyTargetModule

d = DummyTargetModule(targets.VHDL)
walk(d, generator_func, lambda x: x.emit(d), a, b, z)

case a is
when "000001" =>
    b <= "000010";
    z <= '1';
when others =>
    b <= "000000";
    z <= '0';
end case;


## Conclusion

We note: There can be two representations of a Python routine. Since Python does neither allow us to use custom If..Else classes in its native dialect, nor can we override an assignment of a variable, we are a priori required to explicitely write down a dual representation for the same thing, if we want to execute and generate. The other, context sensitive option, is to use a decorator for a routine that does the following:
* AST translation within a generator context to a generator representation using `yield`
* Untouched execution of the function when in a native python execution context

## The myIRL internal representation

When decorating a top level function with a cyhdl `@block` statement, its child funclets will be examined for specific decorator keywords and translation to the **IRL** (internal representation language) will occur. Note that there is a `@myirl.block` wrapper - not to be mixed up - that does not translate, but tag a hardware component in general (like a Verilog module).

This is a static construct and allows to emulate HDL python dialects such as MyHDL up to some extent.

Dynamic selection is provided by specific factory classes that are context sensitive. This means, depending on the target configuration, code is either emit to a HDL target, compiled directly into a hardware simulation or is run natively in conjunction with a simulation. This is also referred to as 'Co-Simulation'.

## HDL macros

In many cases, setting a set of signals repeatedly would call for a function or macro. This is covered by the `@hdlmacro` function or method decorator. It has the following properties:

* From outside, it is called like a function
* Inside, it is using generator notation to create hardware
* it is not context sensitive, but explicit. It can therefore
   not contain context based if construct

In [6]:
class Port:
    def __init__(self, n = 8):
        self.a = Signal(intbv()[n:])
        self.b = Signal(intbv()[n:])
        
    @hdlmacro
    def init(self, v):
        print("Initializing to", v)
        yield [
            self.a.set(v), self.b.set(v)
        ]
        print("Done")

In [7]:
def test_port(p):
    gen = p.init(2)
    return gen

p = Port(5)
g = test_port(p)
p.a.evaluate()

Initializing to 2
Done


intbv(0)

In [8]:
g.evaluate()
assert p.a.evaluate() == 2 and p.b.evaluate() == 2

A pure `@hdlmacro` does not allow to be called using `yield from`.

## When to use yield and when not

Because code is either translated or executed, it is not always transparent in which of the dual forms HDL code should be authored. In some cases it might be desirable to mix native code with procedural constructs in IRL notation.

The IRL provides generator constructs through decorators that work like statements inside a IRL sequence or process. In native form, an undecorated generator functions would be used using `yield from`. Inside a IRL statement list however, a decorated Generator object will be iterated through automatically. This can cause confusion, as shown with the `@hdlmacro` construct below.

First, we must import the augmented `Signal` class from the simulation module. This class is able to evaluate assignments behind the curtains and is thus usable for verification the 'inline' way.

In [9]:
from cyrite.simulation import Signal as SimSignal

@hdlmacro
def genfunc(a, b):
    print("RUN MACRO")
    yield [
        a.set(b),
        b.set(b + 1),
    ]
    print("FINISH MACRO")

Because the `@hdlmacro` is generating expressions, but has not actually evaluated their values yet, we have to explicitely call `evaluate()` in order to see updated values.

We construct a generator sequence that is run *as is*, i.e. untranslated. We refer to this as a **native context**.

In [10]:
def walk_sequence(a, b):
    
    a.next = 0
    b.next = 3
    yield 1
    # @hdlmacro is explicitely *called* and must be evaluated:
    genfunc(a, b).evaluate()
    # This is not allowed:
    try:
        yield from genfunc(a, b)
        assert False
    except RuntimeError:
        pass
    
    # Note the genfunc assignments have
    # not updated yet, because it's a hdlmacro. We must
    # insert a yield timestep:
    assert a == 0 and  b == 3  
    
    yield 1
   
    assert a == 3 and b == 4
    
    print("STOP")

To iterate over this generator, we use the DummySimulator classes `handle_sequence()` method. This simply updates the simulator signals to their `.next` assignment when a time step is taken via `yield`.

In [11]:
from myirl.targets.dummy import DummySimulator

sim = DummySimulator()

a, b = [ sim.Signal(intbv()[4:], name = n) for n in "ab" ]

sim.handle_sequence(walk_sequence(a, b))


[ STEP: 1 -> 1 ]
RUN MACRO
FINISH MACRO
RUN MACRO
FINISH MACRO
[ STEP: 1 -> 2 ]
STOP


The next thing to take notice: `genfunc` is a true hardware generator and requires evaluation in the simulation context. Otherwise, the assertion will fail.

Note that a hardware generator will pass an explicit context to `.evaluate()`, whereas a native caller may omit this.

The `Macro` class behind the `@hdlmacro` decorator prohibits to use `yield from` from within a native context. The reason is, that the native context should only see time steps as yielded expression.

However, when we wish to generate code for external simulators, the game is different: In this case `walk_sequence()` would need to be a generator function and we might have to explicitely rewrite it - if we did not have an auxiliary: We can use said decorators to silently turn it into a generator under the hood.
This context situation is referred to as **generator context** where decorated funclets are translated to the IRL representation.

## Code contexts

We can create decorators with dual use functionality, such that a function is *executed* in one context or *generates* in the other.

A hardware function, for example a IRL `@process`, always runs in parallel with other processes. A simulation specific sequence of stimuli again runs like a program with wait/delay statements.

A `@hdlmacro` is a priori a hardware-minded function which does not describe a timed simulation sequence, therefore it should be emphasized again that it is a pure and explicit generator. When called within a sequential simulation context, it must be evaluated.

**Note**: A `@hdlmacro` is not a portable construct. Never call it from a dual use `@cyrite_method`.

## CAVEATS
 
We also elaborate on the `@hdlmacro` pitfall again, when forgetting to evaluate:

In [12]:
def sequence():
    a, b = [ sim.Signal(intbv()[4:], name = n) for n in "ab" ]
    a.next = 4
    b.next = 2
    yield 1
    assert a == 4 and b == 2 # Values valid after time step!
    ret = genfunc(a, b)
    try:
        assert a == 2 and b == 3 # Expect failure
    except AssertionError:
        print("assertion FAILED, re-evaluate")
        ret.evaluate()
        yield 1 # And step to settle signals
        
    # Try again:
    assert a == 2 and b == 3 # Values valid after evaluation
    

sim.handle_sequence(sequence())

[ STEP: 1 -> 3 ]
RUN MACRO
FINISH MACRO
assertion FAILED, re-evaluate
[ STEP: 1 -> 4 ]


So: only after an explicit evaluation, the action is executed.
In case it is really meant to call an uncedorated hardware related function from a native sequence, you may want to use the following construct (note this is a very rarely used, internal construct for some outdated context implementations):

In [13]:
def bare_dual_func(a, b, v):
    a.next = b
    b.next = v
    yield [ a.set(b), b.set(v) ]

def native_seq():
    assert bare_func(a, b, 3) == None

In this case you will not miss a generator being returned by the called function.

**Note**: When you really **must** call a `@hdlmacro` from a  `@cyrite_method` function, you *must* obtain the current context and pass it to `.evaluate()` in an out-of-band helper function called by the `@cyrite_method` function.

See also [Out of band tricks](#Out-of-band-tricks)

## Portability

A decorated function may be written such that it is valid for both the native and the generator context. Thus, a function object is created that is context sensitive, decides which variant to iterate through or call depending on the context.

This leads to portability issues, as return values of functions can have different meanings in the dual context world.

## Thumb rules

* Avoid to insert `simulation.wait` statements into `@hdlmacro` functions, if you intend to co-simulate.

### Macros, sequences, functions

A native context is not passed automatically to a dual use funclet. A RTL function (not generator) must be `.run` explicitely by passing a context argument in order to actually execute.
Since this is not portable to the generator context, the main rule was distilled as follows:

* **Always** call by `yield from` from a sequential simulation context (even if they contain no `yield` statements):
   * `@rtl_function`
   * `cyrite_method.sequence`
   * `cyrite_method.function`    
   In this case, the generator context is None and the internal  
   wrapper must deal with it.

* call as a function from a pure hardware generation context, i.e.
  inside a `@always()` process. 
   * `@rtl_function`
   * `cyrite_method.function`
   * `@hdlmacro`

`@hdlmacros` as non-portable constructs are thus called by hardware generators, typically.
 
When a hdl macro is called from within a dual use function without evaluation, it may render the dual use function unportable, i.e. such a construct will only work for transpiled output. Some advanced context wrappers may be able to process nested alternating contexts, but typically, you will get an error.

## Code verification - externally

The Cyrite emulation layer provides a `@sequence` decorator to automatically translate a sub-function into a generator.

However, there's a catch: When simply copying the `walk_sequence()` function, an external simulator will fail. Why? When assigning a `Signal` to a value, the value is not immediately valid, rather, a delta time step will have to be taken in order for the signal to update.

Therefore, we have to insert a `delay()` statement before each time we make use of a signals content. We construct a test bench `@block` with an adapted `@sequence`:

In [14]:
from cyhdl import *

@block
def testbench():
    a, b = [ Signal(intbv()[4:], name = n) for n in "ab" ]

    @sequence
    def main():
        a.next = 0
        b.next = 3
        
        yield delay(1) # Wait to settle
        
        # @hdlmacro is explicitely *called*
        genfunc(a, b) # non-portable!

        yield delay(1) # Again, wait to settle signal values
        print(a, b)
        
        assert a == 3
        assert b == 4

        print("STOP")

    return instances()

Note that we can use the `yield from` notation here. It is silently translated into a call inside the resulting yield sequence. We can display this by calling `.unparse()`:

In [15]:
print(testbench.unparse())

Unparsing unit testbench


@block
def testbench():
    (a, b) = [Signal(intbv()[4:], name=n) for n in 'ab']

    @generator_ctx
    def main(_context):
        (yield [a.set(0), b.set(3), wait(delay(1)), genfunc(a, b), wait(delay(1)), print_(a, b), assert_((a == 3), 'Failed in /tmp/ipykernel_3944/4070373390.py:testbench():20'), assert_((b == 4), 'Failed in /tmp/ipykernel_3944/4070373390.py:testbench():21'), print_('STOP')])
    return instances()



In [16]:
from myirl.test.common_test import Simulator
tb = testbench()

s = Simulator(targets.VHDL)
s.run(tb, 200, debug = True)

RUN MACRO
FINISH MACRO
 Writing 'testbench' to file /tmp/testbench.vhdl 
 Creating library file /tmp/module_defs.vhdl 
WORK DIR of instance [Instance testbench I/F: [// ID: testbench_0 to testbench]] /tmp/myirl_testbench_e9i7man4/
==== COSIM stdout ====
0x3 0x4
STOP



0

The simulation hence passes, however, try removing a `delay()` statement and it will fail. This behaviour is always wanted in the simulation world. Because our simulation signals are initialized by a dummy simulation signal internally and receive their assignments immediately, the behaviour is *not* the same.

Finally, we may want to have a look at the transpiled resulting VHDL file:

In [17]:
!cat {s.used_files[0]}

-- File generated from source:
--     /tmp/ipykernel_235/4070373390.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity testbench is
end entity testbench;

architecture cyriteHDL of testbench is
    -- Local type declarations
    -- Signal declarations
    signal a : unsigned(3 downto 0);
    signal b : unsigned(3 downto 0);
begin
    
main:
    process
    begin
        a <= x"0";
        b <= x"3";
        wait for 1 ns;
        a <= b;
        b <= resize((resize(b, 5) + 1), 4);
        wait for 1 ns;
        print("0x"& hstr(a) & " " & "0x"& hstr(b));
        assert (a = x"3")
            report "Failed in /tmp/ipykernel_235/4070373390.py:testbench():20" severity failure;
        assert (b = x"4")
            report "Failed in /tmp/ipykernel_235/4070373390.py:testbench():21" severity failure;
    

### Hardware generation versus simulation

The classical HDL languages don't explicitely differentiate between constructs that translate to real hardware elements or are mainly meant for simulation.

One could thus be tempted to insert `delay()` specifications into a `@hdlmacro` generator in order to simplify the main() sequence.
However this is a deprecated method, because a `@hdlmacro`is particularely meant to generate a hardware construct.

A Python simulation routine however makes the distinction between time steps and events versus hardware elements by the `yield` statement. To keep hardware only and simulation constructs separate, specific decorators are used for sequential statements, such as `@cyrite_method.sequence`. See also [Port classes](ports.ipynb) for examples.

## Out of band tricks

For all functions with a dual use nature, context sensitivity is typically carried out by the transpiler.

In some cases however one might insert special hooks depending on the context. Since a `@cyrite_method` function called from a generator context is turned into a generator as well, we need to except commands that are not emitted to a target structure but are executed. This is referred to as *out of band* code.

A IRL representation for those dual bands looks as follows: The `print` statement is executed and explicitely not emitted.

In [18]:
@hdlmacro
def gen(a):
    print("Inverting", a)
    yield [ a.set(~a) ]

A sequential function construct represented in CyHDL notation and interpreted in dual ways however can not make use of that. We have to create an out of band @hdlmacro that just yields an empty list (we must not yield `None`):

In [19]:
@hdlmacro
def report(msg):
    print(msg)
    yield []

Then we call this from a dual `@seq_function`:

In [20]:
from myirl.emulation.factory_wrapper import seq_function

@seq_function
def test_sequence(a, b):
    a.next = b
    yield delay(1)
    report("Set %s to %s" % (repr(a), repr(b)))

We create a test bench to emit to a HDL language:

In [21]:
from cyhdl import *

@block
def tb():
    a, b = [ Signal(intbv()[5:]) for _ in range(2) ]
    @sequence
    def main():
        a.next = 5
        b.next = 0
        yield from test_sequence(a, b)
        assert a == b
        raise StopSimulation
    
    return instances()

### HDL output

Finally, we elaborate into a HDL. The `@hdlmacro` is iterated upon evaluation, in this case at the elaboration time:

In [22]:
t = tb()

In [23]:
files = t.elab(targets.VHDL)

Set <a> to <b>
 Writing 'tb' to file /tmp/myirl_tb_9ae4kzi7/tb.vhdl 


And we observe the report command not being emitted to the target HDL:

In [24]:
!grep -4 process {files[0]}

    signal b : unsigned(4 downto 0);
begin
    
main:
    process
    begin
        a <= "00101";
        b <= "00000";
        a <= b;
--
        assert (a = b)
            report "Failed in /tmp/ipykernel_235/1687103223.py:tb():11" severity failure;
        std.env.stop;
        wait;
    end process;
end architecture cyriteHDL;

